In [8]:
from flask import Flask
from flask import request
import datetime
from dateutil.parser import parse
import json
import numpy as np
import random 
import collections
import copy 
import math
from operator import itemgetter
import plotly.figure_factory as ff

In [7]:
info_pr = {'process_info': [[[[1, 1, 10], [2, 1, 11]], [[3, 1, 12]], [[2, 1, 13]]],
                            [[[3,1,14], [1,1,15], [2,1,16]], [[2,1,17], [3,1,18]], [[3,1,19]]],
                            [[[1,1,1],[2,1,2]]]],
           "job_nb": 3, 'total_op_nb': 7, 'machine_nb': 3,'machine_list':[1,2,3], 'machine_aval':[0,0,0],
           'start_date':"2022-11-01 00:00:00",
           'job_dict':{1:['order 1', 1], 2:['order 2', 1],3:['order 3', 1]},
           'machine_dict':{1:'machine 1',2:'machine 2',3:'machine 3'}
           }

app = Flask(__name__)


NameError: name 'Flask' is not defined

In [9]:
required_data = {
    "periodStartDate": "2022-11-01 00:00:00", ##! 当前排产周期的开始时间
    "orders": [ ##! 当前排产周期订单信息
        {   
            'orderID': '001',
            'productID': '0213配油盘',
            'productNB': 1
        },
        {
            'orderID': '002',
            'productID': '12M26机体',
            'productNB': 2
        },
        {
            'orderID': '003',
            'productID': 'PSI机体',
            'productNB': 2
        }
    ],
    "process": { ##! 产品加工工艺信息
        '0213配油盘': [
            {
                'machine': ['MC6000-7'],
                'processTime': [20],
                'fixture': ['F1'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [35],
                'fixture': ['F1'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [30],
                'fixture': ['F2'],
                'isInspetion': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [30],
                'fixture': ['F2'],
                'isInspetion': False
            }
        ],
        '12M26机体': [
            {
                'machine': ['MC6000-6'],
                'processTime': [50],
                'fixture': ['F3'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-7'],
                'processTime': [75],
                'fixture': ['F4'],
                'isInspection': False
            },
            {
                'machine': ['MC6000-6'],
                'processTime': [60],
                'fixture': ['F4'],
                'isInsepction': False
            },
            {
                'machine': ['MC6000-6'],
                'processTime': [30],
                'fixture': ['F5'],
                'isInspection': True
            }
        ],
        'PSI机体': [
            {
                'machine': ['德玛吉五轴', '乌尼恩-1#'],
                'processTime': [276, 276],
                'fixture': ['F6', 'F7'],
                'isInspection': False
            },
            {
                'machine': ['德玛吉五轴', '英赛-1#'],
                'processTime': [360, 360],
                'fixture': ['F8', 'F9'],
                'isInspection': False
            },
            {
                'machine': ['德玛吉五轴', '英赛-1#', 'BW'],
                'processTime': [276, 276, 300],
                'fixture': ['F10', 'F11', 'F12'],
                'isInspection': False
            },
            {
                'machine': ['南MTM清洗机'],
                'processTime': [252],
                'fixture': [],
                'isInspection': False
            }
        ]
    },
    "machines": ['MC6000-6', 'MC6000-7', '德玛吉五轴', '乌尼恩-1#', '英赛-1#', 'BW', '南MTM清洗机'], ##! 当前排产周期的设备列表
    "stuff": ['Jack', 'Tom', 'Mary', 'Lily', 'Bob', 'Alice', 'David', 'Eve', 'Frank', 'Grace', 'Helen', 'Ivan', 'John', 'Kevin', 'Lucy', 'Mike', 'Nancy', 'Oscar', 'Peter', 'Queen', 'Robin', 'Steve', 'Tony', 'Uma', 'Victor', 'Wendy', 'Xavier', 'Yvonne', 'Zoe'], ##! 当前排产周期的员工列表
    "machineAval": [0, 0, 0, 0, 0, 0, 0], ##! 当前排产周期的设备可用时间
    "machineNotAvalPeriod": { ##! 当前排产周期的设备不可用时间段
        'MC6000-6': [['2022-11-01 01:00:00', '2022-11-02 08:00:00'], ['2022-11-02 19:00:00', '2022-11-03 08:00:00']], ##! 夜班不可用
        'MC6000-7': [['2022-11-01 19:00:00', '2022-11-02 08:00:00'], ['2022-11-02 19:00:00', '2022-11-03 08:00:00']],
        '德玛吉五轴': [],
        '乌尼恩-1#': [],
        '英赛-1#': [],
        'BW': [],
        '南MTM清洗机': [['2022-11-01 12:00:00', '2022-11-01 18:00:00']] ##! 维修时间
    },
    "nonProcessingTime": 15, ##! 上下料、换夹具等非加工时间
}

In [10]:
def time_transform(start_time: str, period: list): 
    start_time = datetime.datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S")
    target = []
    if bool(period):
        for i in period:
            date1 = parse(i[0])
            date2 = parse(i[1])
            result1 = date1 - start_time
            result2 = date2 - start_time
            target.append([int(result1.total_seconds()//60), int(result2.total_seconds()//60)])
    return target

In [11]:
def info_pr_with_json(data_json):
    global info_pr 
    info_pr['machine_nb'] = len(data_json['machines'])
    dict_machine = {}
    info_pr['machine_dict'] = {}
    for index, machine_name in enumerate(data_json['machines']):
        dict_machine[machine_name] = index + 1
        info_pr['machine_dict'][index+1] = machine_name
    info_pr['machine_list'] = [i for i in range(1, info_pr['machine_nb'] + 1)]
    
    job_nb = 0
    for order in data_json['orders']:
        job_nb += order['productNB']
    info_pr['job_nb'] = job_nb
    info_pr['start_date'] = data_json['periodStartDate']
    info_pr['machineNotAvalPeriod'] = []
    for machine, machine_index in dict_machine.items():
        info_pr['machineNotAvalPeriod'].append(time_transform(data_json['periodStartDate'], data_json['machineNotAvalPeriod'][machine]))
    
    info_pr['process_info'] = []
    info_pr['total_op_nb'] = 0
    job_count = 1
    for order in data_json['orders']:
        for job_index_in_order in range(order['productNB']):
            info_pr['job_dict'][job_count] = [order['orderID'] , job_index_in_order + 1]
            job_count += 1
            job_info = []
            processes = data_json['process'][order['productID']]
            info_pr['total_op_nb'] += len(processes)
            for process in processes:
                process_info = []
                for index, machine in enumerate(process['machine']):
                    temp = [0,1,0]
                    # 设备index 赋值
                    temp[0] = dict_machine[machine]
                    ##! 暂时忽略夹具
                    # 加工时间赋值
                    temp[2] = process['processTime'][index]
                    process_info.append(temp)
                job_info.append(process_info)
            info_pr['process_info'].append(job_info)
    info_pr['machine_aval'] = data_json['machineAval']
    info_pr['nonProcessingTime'] = data_json['nonProcessingTime']
    return info_pr
    

In [12]:
res = info_pr_with_json(required_data)
print(res['job_nb'])
print(res['machine_dict'])
print(res['total_op_nb'])
print(res['machineNotAvalPeriod'])


5
{1: 'MC6000-6', 2: 'MC6000-7', 3: '德玛吉五轴', 4: '乌尼恩-1#', 5: '英赛-1#', 6: 'BW', 7: '南MTM清洗机'}
20
[[[60, 1920], [2580, 3360]], [[1140, 1920], [2580, 3360]], [], [], [], [], [[720, 1080]]]


In [13]:
JOB = [i for i in range(1, res['job_nb'] + 1)]
OPERATION = {}
for i in JOB:
    OPERATION[i] = [i for i in range(1, 1+ len(res['process_info'][i-1]))]
MACHINE = {}
pt = {}
PT = {}
for i in JOB:
    for j in OPERATION[i]:
        MACHINE[(i,j)] = [res['process_info'][i-1][j-1][k][0] for k in range(len(res['process_info'][i-1][j-1]))]
        pt[(i,j)] = [res['process_info'][i-1][j-1][k][2] for k in range(len(res['process_info'][i-1][j-1]))]
        dic_ref = dict(zip(MACHINE[(i,j)], pt[(i,j)]))
        for k in MACHINE[(i,j)]:
            PT[(i,j,k)] = dic_ref[k]
INDEX_DIC = {}
lengh = 0
for i in JOB:
    for j in OPERATION[i]:
        INDEX_DIC[(i,j)] = lengh + OPERATION[i].index(j) + 1
    lengh += len(OPERATION[i])

print(PT)


{(1, 1, 2): 20, (1, 2, 2): 35, (1, 3, 2): 30, (1, 4, 2): 30, (2, 1, 1): 50, (2, 2, 2): 75, (2, 3, 1): 60, (2, 4, 1): 30, (3, 1, 1): 50, (3, 2, 2): 75, (3, 3, 1): 60, (3, 4, 1): 30, (4, 1, 3): 276, (4, 1, 4): 276, (4, 2, 3): 360, (4, 2, 5): 360, (4, 3, 3): 276, (4, 3, 5): 276, (4, 3, 6): 300, (4, 4, 7): 252, (5, 1, 3): 276, (5, 1, 4): 276, (5, 2, 3): 360, (5, 2, 5): 360, (5, 3, 3): 276, (5, 3, 5): 276, (5, 3, 6): 300, (5, 4, 7): 252}


In [14]:
OS = [1,1,5,1,1,2,2,4,4,5,5,2,2,3,3,4,3,3,4,5]
MA = [2,2,2,2,1,2,1,1,1,2,1,1,3,3,6,7,4,3,5,7]
machine_num = res['machine_nb']
nonProcessingTime = res['nonProcessingTime']

def get_OS_index(OS):
    OS_copy = OS.copy()
    OS_index = [] ##! OS中各工序对应在MA中的位置, 从1开始
    OS_index_PT = [] ##! OS中各工序对应该产品的第几道工序，PT:product
    L = len(OS)
    for i in range(L):
        count = collections.Counter(OS_copy)
        operation_index  = len(OPERATION[OS[i]]) - count[OS[i]]
        OS_index.append(INDEX_DIC[(OS[i], OPERATION[OS[i]][operation_index])])
        OS_index_PT.append(OPERATION[OS[i]][operation_index])
        OS_copy.pop(0)
    return OS_index, OS_index_PT

def find_available_gap(notAvalPeriod, pt, nonProcessingTime, last_op_end_time):
    ##! notAvalPeriod: 机器不可用时间段
    ##! pt: 工序加工时间
    ##! nonProcessingTime: 机器间隔时间
    ##! last_op_end_time: 上道工序结束时间
    ##! 返回值：可用时间段的起始时间
    possible_start_time = []
    time_list = notAvalPeriod.copy()
    time_list.sort() ##! 按时间先后排序
    time_list.insert(0, [0,0])
    for i in range(len(time_list)-1):
        gap = time_list[i+1][0] - max(time_list[i][1], last_op_end_time)
        if gap >= pt + nonProcessingTime:
            if last_op_end_time == 0:
                possible_start_time.append(time_list[i][1])
            else:
                possible_start_time.append(max(time_list[i][1], last_op_end_time + nonProcessingTime))
    if possible_start_time == []:
        if last_op_end_time == 0:
            possible_start_time.append(time_list[-1][1])
        else:
            possible_start_time.append(max(time_list[-1][1], last_op_end_time + nonProcessingTime))
    return possible_start_time

def decode(OS_chrom, MA_chrom):
    L = len(OS_chrom)
    schedule = np.zeros((2, L), dtype=int)
    machineNotAvalPeriod = res['machineNotAvalPeriod']
    updated_machineNotAvalPeriod = copy.deepcopy(machineNotAvalPeriod)
    OS_index, OS_index_PT = get_OS_index(OS_chrom)
    for i in range(L):
        current_product = OS_chrom[i]
        current_machine = MA_chrom[OS_index[i]-1] ##! 从1开始
        if OS_index_PT[i] == 1:
            last_op_end_time = 0
        else:
            last_op_end_time = schedule[1][OS_index[i]-2]
        schedule[0][OS_index[i]-1] = find_available_gap(updated_machineNotAvalPeriod[current_machine-1], \
        PT[(current_product, OS_index_PT[i], current_machine)], nonProcessingTime, last_op_end_time)[0]
        schedule[1][OS_index[i]-1] = schedule[0][OS_index[i]-1] + PT[(current_product, OS_index_PT[i], current_machine)]
        ## 更新机器不可用时间段
        updated_machineNotAvalPeriod[current_machine-1].append([schedule[1][OS_index[i]-1], schedule[1][OS_index[i]-1] + nonProcessingTime])
    return schedule

print(decode(OS, MA))

[[   0   35   85  130 1920 1985 2075 2150 1985 2075 2195 2270    0  291
   666 1080    0  666 1041 1347]
 [  20   70  115  160 1970 2060 2135 2180 2035 2150 2255 2300  276  651
   966 1332  276 1026 1317 1599]]


In [15]:
def gannt():
    scheduling = decode(OS,MA)
    X_ijk = {}
    ST_ij = {}
    CT_ij = {}
    for i in JOB:
        for j in OPERATION[i]:
            for k in MACHINE[(i,j)]:
                if k == MA[INDEX_DIC[(i,j)]-1]:
                    X_ijk[(i,j,k)] = 1
                else:
                    X_ijk[(i,j,k)] = 0
    for i in JOB:
        for j in OPERATION[i]:
            ST_ij[(i,j)] = scheduling[0][INDEX_DIC[(i,j)]-1]
            CT_ij[(i,j)] = scheduling[1][INDEX_DIC[(i,j)]-1]
    index_record = []
    job_record = {}
    SolutionDict = {}
    for i in JOB:
        for j in OPERATION[i]:
            for m in MACHINE[(i,j)]:
                Keyname_X = '_(' + str(i) + ',_'+str(j) + ',_' + str(m) + ')'
                SolutionDict[Keyname_X] = [X_ijk[(i,j,m)], ST_ij[(i,j)], CT_ij[(i,j)]]
                if SolutionDict[Keyname_X][0] == 1:
                    initial_time = datetime.datetime.strptime("2022-11-01 00:00:00", "%Y-%m-%d %H:%M:%S")
                    start_time = initial_time + datetime.timedelta(minutes=SolutionDict[Keyname_X][1].astype(np.float64))
                    end_time = initial_time + datetime.timedelta(minutes=SolutionDict[Keyname_X][2].astype(np.float64))
                    processing_time = CT_ij[(i,j)] - ST_ij[(i,j)]
                    job_record[(i,j,m)] = [start_time,end_time,processing_time]
                    index_record.append([i,j,m])
    index_record.sort(key=itemgetter(2))
    df1 = []
    colors = {}
    for i in index_record:
        df1.append(dict(Task='Machine %s' % (i[2]), Start='%s' % (str(job_record[(i[0],i[1],i[2])][0])),\
            Finish='%s' % (str(job_record[(i[0],i[1],i[2])][1])),\
                Resource='Job %s' % (i[0])))
    for i in JOB:
        color_list = [random.randint(0,255),random.randint(0,255),random.randint(0,255)]
        colors['Job %s' % i] = '#%02X%02X%02X' % (color_list[0],color_list[1],color_list[2])
    fig1 = ff.create_gantt(df1, index_col='Resource', colors=colors, show_colorbar=True, group_tasks=True, showgrid_x=True,\
        title='gannt chart')
    fig1.write_html('test' + '.html')
    print("gannt chart finished")

In [16]:
gannt = gannt()

gannt chart finished
